In [2]:
import pandas as pd
import numpy as np
import cv2, os
import csv

# libraries needed from outside
import os
from pathlib import Path
from os import mkdir
from os.path import exists, join, basename, splitext
import sys
import glob                       # help for finding files
from re import split              # regular expression string splitter
import subprocess                 #subprocess wraps around regular os commands
#import shutil                     # used for copying files in the os.
import pandas as pd
!pip install validators
import validators                 #for URL validation
#from validators import ValidationFailure


from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# get pose data - data is generated by open pose video
#check_url(input_csv)

df = pd.read_csv("https://raw.githubusercontent.com/eeroolli/SalsaAnnotation/development/ml/models/Data_train_validate/Data_train_norm_1.csv")

df_original = df.copy()
df_original = df.drop(columns='Unnamed: 0')
df_original

,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,1p_Eero_7_05122021_choreo1,2047,0.0,0.637984,0.774014,0.920893,0.334503,0.893144,0.404510,0.873358,0.450967,0.539857,0.618345,0.788832,0.372212,0.666264,0.323270,0.511786,0.629526,0.932929,0.278045,0.939904,0.186849,0.845104,0.373757,0.806770,0.491059,0.857033,0.315991,0.755816,0.377458,0.742389,0.476913,0.291916,0.895899,0.772782,0.706721,0.734656,0.676064,0.812948,...,0.654235,0.377261,0.656807,0.445003,0.759317,0.541939,0.821538,0.476642,0.833152,0.473639,0.838034,0.518715,0.853415,0.879803,0.803364,0.157903,0.282602,0.884895,0.946747,0.828071,0.714274,0.712089,0.838662,0.789613,0.665587,0.852450,0.831530,0.0,0.814348,0.000000,0.914645,0.768292,0.793404,0.795577,0.356110,0.250683,0.743280,basic,S,220
1,1p_Eero_7_05122021_choreo1,2048,0.0,0.643427,0.781177,0.920296,0.367403,0.893300,0.398000,0.900727,0.443204,0.569765,0.601303,0.782895,0.406182,0.637963,0.327354,0.464253,0.578958,0.932508,0.284967,0.940839,0.183805,0.837313,0.337844,0.819524,0.459437,0.854966,0.331098,0.764040,0.382883,0.744288,0.473583,0.292479,0.896503,0.780903,0.707201,0.739562,0.671188,0.816883,...,0.650069,0.383648,0.652763,0.445517,0.759996,0.538270,0.828423,0.459908,0.842267,0.455602,0.851191,0.484862,0.822891,0.882205,0.829893,0.852556,0.179996,0.845847,0.936913,0.877548,0.735435,0.739755,0.831251,0.816222,0.678062,0.862401,0.823828,0.0,0.780454,0.000000,0.920393,0.772300,0.789752,0.812520,0.576338,0.469384,0.722062,basic,NaN,220
2,1p_Eero_7_05122021_choreo1,2049,0.0,0.654291,0.783248,0.920237,0.400327,0.895089,0.415802,0.924771,0.416943,0.603704,0.587172,0.775781,0.450690,0.610223,0.327647,0.430420,0.527538,0.933219,0.301048,0.941352,0.187249,0.832612,0.316338,0.832563,0.442637,0.858418,0.375091,0.773300,0.388966,0.746211,0.471582,0.293043,0.897108,0.794700,0.701190,0.744231,0.663293,0.821476,...,0.645016,0.388075,0.649356,0.446063,0.760512,0.529752,0.838564,0.443344,0.851691,0.439839,0.863607,0.465854,0.842273,0.884259,0.832634,0.828490,0.296463,0.827820,0.903506,0.799872,0.728836,0.738614,0.831048,0.790879,0.681409,0.793449,0.811598,0.0,0.789451,0.000000,0.918237,0.773667,0.788074,0.807926,0.701255,0.589643,0.702901,basic,NaN,220
3,1p_Eero_7_05122021_choreo1,2050,0.0,0.667740,0.777788,0.920743,0.430589,0.900785,0.439379,0.944034,0.371205,0.639189,0.584571,0.768819,0.484939,0.584386,0.317624,0.412622,0.471199,0.934245,0.325600,0.941519,0.184957,0.833800,0.314625,0.843218,0.435421,0.861367,0.435699,0.781171,0.396960,0.747617,0.468924,0.293606,0.897712,0.810855,0.696196,0.749070,0.659945,0.828762,...,0.642808,0.389963,0.647433,0.445615,0.760920,0.524253,0.846664,0.434850,0.859269,0.434421,0.875831,0.457193,0.790833,0.889678,0.826872,0.860735,0.408687,0.799390,0.944180,0.881825,0.737036,0.740039,0.819579,0.779000,0.691094,0.781239,0.813263,0.0,0.770174,0.000000,0.916075,0.794719,0.822523,0.826437,0.707142,0.631278,0.689080,basic,NaN,220
4,1p_Eero_7_05122021_choreo1,2051,0.0,0.680521,0.765225,0.922279,0.468861,0.910224,0.483731,0.965584,0.363131,0.671361,0.603732,0.762912,0.514193,0.566384,0.295988,0.404856,0.427584,0.934310,0.351438,0.941269,0.177319,0.839073,0.320470,0.851596,0.436101,0.867249,0.475815,0.785576,0.401853,0.747995,0.466996,0.294169,0.898316,0.828934,0.691208,0.754343,0.660513,0.839789,...,0.642497,0.387622,0.646009,0.443139,0.760501,0.51921

In [ ]:
df.drop(columns='Unnamed: 0')

,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,1p_Eero_7_05122021_choreo1,2047,0.0,0.637984,0.774014,0.920893,0.334503,0.893144,0.404510,0.873358,0.450967,0.539857,0.618345,0.788832,0.372212,0.666264,0.323270,0.511786,0.629526,0.932929,0.278045,0.939904,0.186849,0.845104,0.373757,0.806770,0.491059,0.857033,0.315991,0.755816,0.377458,0.742389,0.476913,0.291916,0.895899,0.772782,0.706721,0.734656,0.676064,0.812948,...,0.654235,0.377261,0.656807,0.445003,0.759317,0.541939,0.821538,0.476642,0.833152,0.473639,0.838034,0.518715,0.853415,0.879803,0.803364,0.157903,0.282602,0.884895,0.946747,0.828071,0.714274,0.712089,0.838662,0.789613,0.665587,0.852450,0.831530,0.0,0.814348,0.000000,0.914645,0.768292,0.793404,0.795577,0.356110,0.250683,0.743280,basic,S,220
1,1p_Eero_7_05122021_choreo1,2048,0.0,0.643427,0.781177,0.920296,0.367403,0.893300,0.398000,0.900727,0.443204,0.569765,0.601303,0.782895,0.406182,0.637963,0.327354,0.464253,0.578958,0.932508,0.284967,0.940839,0.183805,0.837313,0.337844,0.819524,0.459437,0.854966,0.331098,0.764040,0.382883,0.744288,0.473583,0.292479,0.896503,0.780903,0.707201,0.739562,0.671188,0.816883,...,0.650069,0.383648,0.652763,0.445517,0.759996,0.538270,0.828423,0.459908,0.842267,0.455602,0.851191,0.484862,0.822891,0.882205,0.829893,0.852556,0.179996,0.845847,0.936913,0.877548,0.735435,0.739755,0.831251,0.816222,0.678062,0.862401,0.823828,0.0,0.780454,0.000000,0.920393,0.772300,0.789752,0.812520,0.576338,0.469384,0.722062,basic,NaN,220
2,1p_Eero_7_05122021_choreo1,2049,0.0,0.654291,0.783248,0.920237,0.400327,0.895089,0.415802,0.924771,0.416943,0.603704,0.587172,0.775781,0.450690,0.610223,0.327647,0.430420,0.527538,0.933219,0.301048,0.941352,0.187249,0.832612,0.316338,0.832563,0.442637,0.858418,0.375091,0.773300,0.388966,0.746211,0.471582,0.293043,0.897108,0.794700,0.701190,0.744231,0.663293,0.821476,...,0.645016,0.388075,0.649356,0.446063,0.760512,0.529752,0.838564,0.443344,0.851691,0.439839,0.863607,0.465854,0.842273,0.884259,0.832634,0.828490,0.296463,0.827820,0.903506,0.799872,0.728836,0.738614,0.831048,0.790879,0.681409,0.793449,0.811598,0.0,0.789451,0.000000,0.918237,0.773667,0.788074,0.807926,0.701255,0.589643,0.702901,basic,NaN,220
3,1p_Eero_7_05122021_choreo1,2050,0.0,0.667740,0.777788,0.920743,0.430589,0.900785,0.439379,0.944034,0.371205,0.639189,0.584571,0.768819,0.484939,0.584386,0.317624,0.412622,0.471199,0.934245,0.325600,0.941519,0.184957,0.833800,0.314625,0.843218,0.435421,0.861367,0.435699,0.781171,0.396960,0.747617,0.468924,0.293606,0.897712,0.810855,0.696196,0.749070,0.659945,0.828762,...,0.642808,0.389963,0.647433,0.445615,0.760920,0.524253,0.846664,0.434850,0.859269,0.434421,0.875831,0.457193,0.790833,0.889678,0.826872,0.860735,0.408687,0.799390,0.944180,0.881825,0.737036,0.740039,0.819579,0.779000,0.691094,0.781239,0.813263,0.0,0.770174,0.000000,0.916075,0.794719,0.822523,0.826437,0.707142,0.631278,0.689080,basic,NaN,220
4,1p_Eero_7_05122021_choreo1,2051,0.0,0.680521,0.765225,0.922279,0.468861,0.910224,0.483731,0.965584,0.363131,0.671361,0.603732,0.762912,0.514193,0.566384,0.295988,0.404856,0.427584,0.934310,0.351438,0.941269,0.177319,0.839073,0.320470,0.851596,0.436101,0.867249,0.475815,0.785576,0.401853,0.747995,0.466996,0.294169,0.898316,0.828934,0.691208,0.754343,0.660513,0.839789,...,0.642497,0.387622,0.646009,0.443139,0.760501,0.51921

In [ ]:
#df.drop(df.columns.str.startswith('alp'))
#df.loc[df.columns.startswith('Unnamed')]
unnamed_list = [df.columns.str.startswith('Unnamed')]
unnamed_list.index(True)

NameError: ignored

In [5]:
clip_names = df_original['clip_name'].unique()


In [6]:
df.columns

Index(['Unnamed: 0', 'clip_name', 'frame_nr', 'person_id', 'nose_x', 'nose_y',
       'neck_x', 'neck_y', 'rshoulder_x', 'rshoulder_y', 'relbow_x',
       'relbow_y', 'rwrist_x', 'rwrist_y', 'lshoulder_x', 'lshoulder_y',
       'lelbow_x', 'lelbow_y', 'lwrist_x', 'lwrist_y', 'midhip_x', 'midhip_y',
       'rhip_x', 'rhip_y', 'rknee_x', 'rknee_y', 'rankle_x', 'rankle_y',
       'lhip_x', 'lhip_y', 'lknee_x', 'lknee_y', 'lankle_x', 'lankle_y',
       'reye_x', 'reye_y', 'leye_x', 'leye_y', 'rear_x', 'rear_y', 'lear_x',
       'lear_y', 'lbigtoe_x', 'lbigtoe_y', 'lsmalltoe_x', 'lsmalltoe_y',
       'lheal_x', 'lheal_y', 'rbigtoe_x', 'rbigtoe_y', 'rsmalltoe_x',
       'rsmalltoe_y', 'rheal_x', 'rheal_y', 'nose_v', 'neck_v', 'rshoulder_v',
       'relbow_v', 'rwrist_v', 'lshoulder_v', 'lelbow_v', 'lwrist_v',
       'midhip_v', 'rhip_v', 'rknee_v', 'rankle_v', 'lhip_v', 'lknee_v',
       'lankle_v', 'reye_v', 'leye_v', 'rear_v', 'lear_v', 'lbigtoe_v',
       'lsmalltoe_v', 'lheal_v', 'rbigto

## Functions

In [7]:

def normalize_clip_by_clip(df_original, df_augmented, col_list):
  # col_list defines which columns to normalize
  # df_original should have all variables
  # df_augmented should have those variables that have been augmented.

  clip_names = df_original['clip_name'].unique()
  df_augmented_n = df_augmented[col_list].copy()   
  for clip_name in clip_names:
    df_augmented_new = pd.DataFrame(normalize(df_augmented_n.loc[df_original['clip_name']==clip_name]))   # the Numpy Array is converted to a Pandas df. no column names.
    df_augmented_n.update(df_augmented_new)                 

  #[df_augmented1["clip_name"==clip_name]]
  df_augmented_n.columns = col_list                    # adds columnnames
  df_augmented_n                                # a numpy array with all changes

  df_augmented_norm = df_original.copy()                # get all the data
  df_augmented_norm.update(df_augmented_n)            # and add the changes
  return df_augmented_norm





In [8]:
# +++++++++++++++++++++++++++++++++++
#    After perspective change we normalize

# this must be run by clip_names
def normalize(array):
  max_value = np.max(array)
  min_value = np.min(array)
  diff = max_value - min_value
  normalized = (array - min_value) / diff
  return normalized



## Flip on X axis

Flip json. Use normalized data x = 1 - x



In [9]:
#df.columns.str.endswith('_x')

In [10]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#          Data Augmentation -- Lists of column names
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++

pose_x_list = ['nose_x', 'neck_x', 'rshoulder_x', 'relbow_x', 'rwrist_x', 'lshoulder_x', 'lelbow_x', 'lwrist_x', 
              'midhip_x', 'rhip_x', 'rknee_x', 'rankle_x', 'lhip_x', 'lknee_x', 'lankle_x', 
              'reye_x', 'leye_x',  'rear_x', 'lear_x',
              'lbigtoe_x', 'lsmalltoe_x', 'lheal_x', 'rbigtoe_x', 'rsmalltoe_x', 'rheal_x' ]

pose_v_list = []
pose_y_list = []
for x_var in pose_x_list:
   pose_y_list.append(x_var.replace('_x', '_y')) 
   pose_v_list.append(x_var.replace('_x', '_v')) 

pose_xy_list = pose_x_list + pose_y_list
print(pose_xy_list)

['nose_x', 'neck_x', 'rshoulder_x', 'relbow_x', 'rwrist_x', 'lshoulder_x', 'lelbow_x', 'lwrist_x', 'midhip_x', 'rhip_x', 'rknee_x', 'rankle_x', 'lhip_x', 'lknee_x', 'lankle_x', 'reye_x', 'leye_x', 'rear_x', 'lear_x', 'lbigtoe_x', 'lsmalltoe_x', 'lheal_x', 'rbigtoe_x', 'rsmalltoe_x', 'rheal_x', 'nose_y', 'neck_y', 'rshoulder_y', 'relbow_y', 'rwrist_y', 'lshoulder_y', 'lelbow_y', 'lwrist_y', 'midhip_y', 'rhip_y', 'rknee_y', 'rankle_y', 'lhip_y', 'lknee_y', 'lankle_y', 'reye_y', 'leye_y', 'rear_y', 'lear_y', 'lbigtoe_y', 'lsmalltoe_y', 'lheal_y', 'rbigtoe_y', 'rsmalltoe_y', 'rheal_y']


In [11]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#          Data Augmentation -- Horisontal Flip
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++

import numpy as np
np.set_printoptions(precision=2)  # For compact display.
df_flipped = df_original.copy()
             

for var in pose_x_list:
  df_flipped[var] = 1 - df_original[var]


In [13]:
print(df_flipped.nose_x.describe())
print(df_original.nose_x.describe())

count    47542.000000
mean         0.475802
std          0.200513
min          0.000000
25%          0.327614
50%          0.483186
75%          0.611720
max          1.000000
Name: nose_x, dtype: float64
count    47542.000000
mean         0.524198
std          0.200513
min          0.000000
25%          0.388280
50%          0.516814
75%          0.672386
max          1.000000
Name: nose_x, dtype: float64


In [14]:
## Saving the results

data_output_path = "/content/gdrive/MyDrive/DanceApp/output_op_1p"
data_output_csv = os.path.join(data_output_path, "Data_train_flipped_x_renorm_x.csv")
df_flipped.to_csv(data_output_csv)




## Perspective change. 

Use normalized data:
Y stays the same.
X gets smaller in the top of the frame. 

Top of the frame Y = 0 
Bottom = 1. If x is reduced by 50% on top of the frame.  Bottom is multiplied with 1-Y. Top is multiplied with 0.5  multiplicator goes from [0, 0.5] when y goes from [1,0]


In [15]:
#+++++++++++++++++++++++++++++++++++++++++++++
#   Data Augmentation - Perspective Change 1
#   Reduce the width on top 
# ++++++++++++++++++++++++++++++++++++++++++++

change_x_factor = 0.4
df_perspective1 = df_original.copy()

for x_var in pose_x_list:
  y_var = x_var.replace('_x', '_y')
  # print(x_var, y_var)
  df_perspective1[x_var] = (1- (change_x_factor * (1- df_original[y_var]))) * df_original[x_var]



In [16]:
print(df_perspective1.nose_x.describe())
print(df_original.nose_x.describe())

count    47542.000000
mean         0.412704
std          0.164716
min          0.000000
25%          0.295495
50%          0.404131
75%          0.526263
max          0.950367
Name: nose_x, dtype: float64
count    47542.000000
mean         0.524198
std          0.200513
min          0.000000
25%          0.388280
50%          0.516814
75%          0.672386
max          1.000000
Name: nose_x, dtype: float64


In [17]:
# normalized by only x.
df_perspective1_x_norm = normalize_clip_by_clip(df_original=df_original, 
                                              df_augmented=df_perspective1, 
                                              col_list=pose_x_list)

data_output_csv = os.path.join(data_output_path, "Data_train_perspective1_renorm_x.csv")
df_perspective1_x_norm.to_csv(data_output_csv)

In [18]:
# normalized by x and y.
df_perspective1_xy_norm = normalize_clip_by_clip(df_original=df_original, 
                                              df_augmented=df_perspective1, 
                                              col_list=pose_xy_list)
data_output_csv = os.path.join(data_output_path, "Data_train_perspective1_renorm_xy.csv")
df_perspective1_xy_norm.to_csv(data_output_csv)

In [21]:

#+++++++++++++++++++++++++++++++++++++++++++++
#   Data Augmentation - Perspective Change 2
#   Reduce the width on top, change Y a bit 
# ++++++++++++++++++++++++++++++++++++++++++++


change_x_factor = 0.3
change_y_factor = np.random.uniform(0.01, 0.02)
df_perspective2 = df_original.copy()

# apply different changes to _x and _y
for x_var in pose_x_list:
  y_var = x_var.replace('_x', '_y')
  # print(x_var, y_var)
  df_perspective2[x_var] =  (1-(change_x_factor * (1- df_original[y_var]))) * df_original[x_var]
  df_perspective2[y_var] =  (1- change_y_factor) * df_original[y_var]
  

# normalized by x and y.
df_perspective2_xy_norm = normalize_clip_by_clip(df_original=df_original, 
                                              df_augmented=df_perspective2, 
                                              col_list=pose_xy_list)
data_output_csv = os.path.join(data_output_path, "Data_train_perspective2_renorm_xy.csv")
df_perspective2_xy_norm.to_csv(data_output_csv)


In [22]:
df_original.head(-10)

,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,1p_Eero_7_05122021_choreo1,2047,0.0,0.637984,0.774014,0.920893,0.334503,0.893144,0.404510,0.873358,0.450967,0.539857,0.618345,0.788832,0.372212,0.666264,0.323270,0.511786,0.629526,0.932929,0.278045,0.939904,0.186849,0.845104,0.373757,0.806770,0.491059,0.857033,0.315991,0.755816,0.377458,0.742389,0.476913,0.291916,0.895899,0.772782,0.706721,0.734656,0.676064,0.812948,...,0.654235,0.377261,0.656807,0.445003,0.759317,0.541939,0.821538,0.476642,0.833152,0.473639,0.838034,0.518715,0.853415,0.879803,0.803364,0.157903,0.282602,0.884895,0.946747,0.828071,0.714274,0.712089,0.838662,0.789613,0.665587,0.852450,0.831530,0.0,0.814348,0.000000,0.914645,0.768292,0.793404,0.795577,0.356110,0.250683,0.743280,basic,S,220
1,1p_Eero_7_05122021_choreo1,2048,0.0,0.643427,0.781177,0.920296,0.367403,0.893300,0.398000,0.900727,0.443204,0.569765,0.601303,0.782895,0.406182,0.637963,0.327354,0.464253,0.578958,0.932508,0.284967,0.940839,0.183805,0.837313,0.337844,0.819524,0.459437,0.854966,0.331098,0.764040,0.382883,0.744288,0.473583,0.292479,0.896503,0.780903,0.707201,0.739562,0.671188,0.816883,...,0.650069,0.383648,0.652763,0.445517,0.759996,0.538270,0.828423,0.459908,0.842267,0.455602,0.851191,0.484862,0.822891,0.882205,0.829893,0.852556,0.179996,0.845847,0.936913,0.877548,0.735435,0.739755,0.831251,0.816222,0.678062,0.862401,0.823828,0.0,0.780454,0.000000,0.920393,0.772300,0.789752,0.812520,0.576338,0.469384,0.722062,basic,NaN,220
2,1p_Eero_7_05122021_choreo1,2049,0.0,0.654291,0.783248,0.920237,0.400327,0.895089,0.415802,0.924771,0.416943,0.603704,0.587172,0.775781,0.450690,0.610223,0.327647,0.430420,0.527538,0.933219,0.301048,0.941352,0.187249,0.832612,0.316338,0.832563,0.442637,0.858418,0.375091,0.773300,0.388966,0.746211,0.471582,0.293043,0.897108,0.794700,0.701190,0.744231,0.663293,0.821476,...,0.645016,0.388075,0.649356,0.446063,0.760512,0.529752,0.838564,0.443344,0.851691,0.439839,0.863607,0.465854,0.842273,0.884259,0.832634,0.828490,0.296463,0.827820,0.903506,0.799872,0.728836,0.738614,0.831048,0.790879,0.681409,0.793449,0.811598,0.0,0.789451,0.000000,0.918237,0.773667,0.788074,0.807926,0.701255,0.589643,0.702901,basic,NaN,220
3,1p_Eero_7_05122021_choreo1,2050,0.0,0.667740,0.777788,0.920743,0.430589,0.900785,0.439379,0.944034,0.371205,0.639189,0.584571,0.768819,0.484939,0.584386,0.317624,0.412622,0.471199,0.934245,0.325600,0.941519,0.184957,0.833800,0.314625,0.843218,0.435421,0.861367,0.435699,0.781171,0.396960,0.747617,0.468924,0.293606,0.897712,0.810855,0.696196,0.749070,0.659945,0.828762,...,0.642808,0.389963,0.647433,0.445615,0.760920,0.524253,0.846664,0.434850,0.859269,0.434421,0.875831,0.457193,0.790833,0.889678,0.826872,0.860735,0.408687,0.799390,0.944180,0.881825,0.737036,0.740039,0.819579,0.779000,0.691094,0.781239,0.813263,0.0,0.770174,0.000000,0.916075,0.794719,0.822523,0.826437,0.707142,0.631278,0.689080,basic,NaN,220
4,1p_Eero_7_05122021_choreo1,2051,0.0,0.680521,0.765225,0.922279,0.468861,0.910224,0.483731,0.965584,0.363131,0.671361,0.603732,0.762912,0.514193,0.566384,0.295988,0.404856,0.427584,0.934310,0.351438,0.941269,0.177319,0.839073,0.320470,0.851596,0.436101,0.867249,0.475815,0.785576,0.401853,0.747995,0.466996,0.294169,0.898316,0.828934,0.691208,0.754343,0.660513,0.839789,...,0.642497,0.387622,0.646009,0.443139,0.760501,0.51921

In [23]:
df_perspective2_xy_norm.head(-10)

,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,1p_Eero_7_05122021_choreo1,2047,0.0,0.784037,0.774014,0.832940,0.334503,0.855492,0.407396,0.837228,0.450967,0.580810,0.618345,0.731848,0.372212,0.644189,0.323270,0.520674,0.629526,0.856896,0.278045,0.880992,0.186849,0.788639,0.373757,0.783893,0.491059,0.769288,0.315991,0.708430,0.377458,0.734312,0.476913,0.403999,0.895899,0.797499,0.714198,0.881183,0.676064,0.802667,...,0.612071,0.377261,0.629907,0.445003,0.745208,0.541939,0.819112,0.476642,0.826273,0.473639,0.809407,0.518715,0.853415,0.879803,0.803364,0.157903,0.282602,0.884895,0.946747,0.828071,0.714274,0.712089,0.838662,0.789613,0.665587,0.852450,0.831530,0.0,0.814348,0.000000,0.914645,0.768292,0.793404,0.795577,0.356110,0.250683,0.743280,basic,S,220
1,1p_Eero_7_05122021_choreo1,2048,0.0,0.792550,0.781177,0.842666,0.367403,0.853607,0.400841,0.861056,0.443204,0.609448,0.601303,0.735461,0.406182,0.617773,0.327354,0.464254,0.578958,0.858781,0.284967,0.880803,0.183805,0.771002,0.337844,0.787370,0.459437,0.771809,0.331098,0.717572,0.382883,0.735317,0.473583,0.404855,0.896503,0.806007,0.714683,0.885630,0.671188,0.807737,...,0.609606,0.383648,0.626145,0.445517,0.744922,0.538270,0.821058,0.459908,0.829944,0.455602,0.812356,0.484862,0.822891,0.882205,0.829893,0.852556,0.179996,0.845847,0.936913,0.877548,0.735435,0.739755,0.831251,0.816222,0.678062,0.862401,0.823828,0.0,0.780454,0.000000,0.920393,0.772300,0.789752,0.812520,0.576338,0.469384,0.722062,basic,NaN,220
2,1p_Eero_7_05122021_choreo1,2049,0.0,0.806467,0.783248,0.852884,0.400327,0.860891,0.418769,0.875680,0.416943,0.642641,0.587172,0.740618,0.450690,0.590977,0.327647,0.422822,0.527538,0.864714,0.301048,0.882490,0.187249,0.760501,0.316338,0.795085,0.442637,0.787720,0.375091,0.727895,0.388966,0.736692,0.471582,0.405710,0.897108,0.818625,0.708608,0.888879,0.663293,0.812175,...,0.605853,0.388075,0.623000,0.446063,0.743217,0.529752,0.826180,0.443344,0.834487,0.439839,0.818646,0.465854,0.842273,0.884259,0.832634,0.828490,0.296463,0.827820,0.903506,0.799872,0.728836,0.738614,0.831048,0.790879,0.681409,0.793449,0.811598,0.0,0.789451,0.000000,0.918237,0.773667,0.788074,0.807926,0.701255,0.589643,0.702901,basic,NaN,220
3,1p_Eero_7_05122021_choreo1,2050,0.0,0.821603,0.777788,0.862799,0.430589,0.873799,0.442515,0.879055,0.371205,0.679809,0.584571,0.743000,0.484939,0.563822,0.317624,0.397355,0.471199,0.873732,0.325600,0.881844,0.184957,0.761094,0.314625,0.803167,0.435421,0.808114,0.435699,0.737463,0.396960,0.737381,0.468924,0.406566,0.897712,0.833892,0.703562,0.893659,0.659945,0.818029,...,0.604198,0.389963,0.621054,0.445615,0.742187,0.524253,0.831609,0.434850,0.840267,0.434421,0.827665,0.457193,0.790833,0.889678,0.826872,0.860735,0.408687,0.799390,0.944180,0.881825,0.737036,0.740039,0.819579,0.779000,0.691094,0.781239,0.813263,0.0,0.770174,0.000000,0.916075,0.794719,0.822523,0.826437,0.707142,0.631278,0.689080,basic,NaN,220
4,1p_Eero_7_05122021_choreo1,2051,0.0,0.833947,0.765225,0.876206,0.468861,0.897079,0.487183,0.896437,0.363131,0.718714,0.603732,0.744945,0.514193,0.541994,0.295988,0.383814,0.427584,0.882284,0.351438,0.878935,0.177319,0.767598,0.320470,0.811346,0.436101,0.825420,0.475815,0.742951,0.401853,0.737246,0.466996,0.407423,0.898316,0.851081,0.698521,0.900120,0.660513,0.829979,...,0.603387,0.387622,0.619136,0.443139,0.740471,0.51921

In [24]:
df_perspective1[pose_x_list]

,nose_x,neck_x,rshoulder_x,relbow_x,rwrist_x,lshoulder_x,lelbow_x,lwrist_x,midhip_x,rhip_x,rknee_x,rankle_x,lhip_x,lknee_x,lankle_x,reye_x,leye_x,rear_x,lear_x,lbigtoe_x,lsmalltoe_x,lheal_x,rbigtoe_x,rsmalltoe_x,rheal_x
0,0.580313,0.675752,0.680401,0.681557,0.457441,0.590744,0.485912,0.435945,0.663516,0.634190,0.633408,0.642531,0.622546,0.567605,0.587055,0.279761,0.682126,0.639463,0.665651,0.491268,0.510997,0.620192,0.649555,0.657737,0.676701
1,0.587109,0.687425,0.678193,0.700118,0.478900,0.596936,0.466313,0.386065,0.665799,0.633676,0.615540,0.642323,0.626211,0.575439,0.587565,0.280371,0.689444,0.642292,0.670255,0.489800,0.507984,0.619631,0.649453,0.658855,0.675799
2,0.597563,0.699501,0.685925,0.709093,0.504014,0.605324,0.446109,0.349077,0.672309,0.635318,0.604922,0.646947,0.643845,0.584295,0.588487,0.280982,0.699714,0.643996,0.673902,0.487135,0.505475,0.617460,0.651847,0.660857,0.679090
3,0.608388,0.711030,0.698785,0.706593,0.532974,0.610423,0.424878,0.325344,0.682223,0.634568,0.605214,0.652792,0.666939,0.592740,0.588800,0.281593,0.712318,0.647180,0.678305,0.485953,0.503862,0.616118,0.655267,0.664875,0.685668
4,0.616613,0.726336,0.722256,0.719604,0.564945,0.614661,0.406888,0.312158,0.691927,0.631523,0.611003,0.659510,0.685409,0.597620,0.588521,0.282205,0.726546,0.651907,0.688574,0.485117,0.502114,0.614246,0.660242,0.668763,0.695012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47537,0.498836,0.465409,0.592756,0.631236,0.509585,0.367628,0.333967,0.363121,0.422029,0.459936,0.440517,0.413749,0.376256,0.335304,0.317156,0.712162,0.511964,0.586509,0.467473,0.318748,0.292372,0.327877,0.387173,0.433102,0.412858
47538,0.500325,0.467792,0.597105,0.632259,0.507430,0.368014,0.328528,0.361913,0.422239,0.463259,0.439876,0.412957,0.379498,0.335224,0.318632,0.711956,0.513302,0.593583,0.474335,0.319558,0.295246,0.330620,0.389630,0.433480,0.412347
47539,0.501815,0.469104,0.602230,0.633211,0.508723,0.369553,0.321491,0.358874,0.422161,0.466791,0.440117,0.412663,0.382097,0.337033,0.322012,0.711750,0.514642,0.598402,0.478021,0.327432,0.301664,0.332740,0.397781,0.436493,0.411340
47540,0.503306,0.472488,0.606883,0.632310,0.507722,0.370753,0.313427,0.356607,0.421683,0.467561,0.440346,0.413490,0.381850,0.340894,0.328381,0.711545,0.515981,0.604252,0.480039,0.340232,0.309268,0.336842,0.409999,0.439669,0.411896


In [32]:
df_original["figure_id"] = df_original['figure_id'] 
df_perspective1["figure_id"] = df_perspective1['figure_id'] + 10000
df_perspective2_xy_norm["figure_id"] = df_perspective2_xy_norm["figure_id"] + 20000

df_augmented = df_perspective1.append(df_perspective2_xy_norm) 
df_augmented = df_augmented.append(df_original)
df_augmented

,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,1p_Eero_7_05122021_choreo1,2047,0.0,0.580313,0.774014,0.675752,0.334503,0.680401,0.404510,0.681557,0.450967,0.457441,0.618345,0.590744,0.372212,0.485912,0.323270,0.435945,0.629526,0.663516,0.278045,0.634190,0.186849,0.633408,0.373757,0.642531,0.491059,0.622546,0.315991,0.567605,0.377458,0.587055,0.476913,0.279761,0.895899,0.682126,0.706721,0.639463,0.676064,0.665651,...,0.491268,0.377261,0.510997,0.445003,0.620192,0.541939,0.649555,0.476642,0.657737,0.473639,0.676701,0.518715,0.853415,0.879803,0.803364,0.157903,0.282602,0.884895,0.946747,0.828071,0.714274,0.712089,0.838662,0.789613,0.665587,0.852450,0.831530,0.0,0.814348,0.000000,0.914645,0.768292,0.793404,0.795577,0.356110,0.250683,0.743280,basic,S,10220
1,1p_Eero_7_05122021_choreo1,2048,0.0,0.587109,0.781177,0.687425,0.367403,0.678193,0.398000,0.700118,0.443204,0.478900,0.601303,0.596936,0.406182,0.466313,0.327354,0.386065,0.578958,0.665799,0.284967,0.633676,0.183805,0.615540,0.337844,0.642323,0.459437,0.626211,0.331098,0.575439,0.382883,0.587565,0.473583,0.280371,0.896503,0.689444,0.707201,0.642292,0.671188,0.670255,...,0.489800,0.383648,0.507984,0.445517,0.619631,0.538270,0.649453,0.459908,0.658855,0.455602,0.675799,0.484862,0.822891,0.882205,0.829893,0.852556,0.179996,0.845847,0.936913,0.877548,0.735435,0.739755,0.831251,0.816222,0.678062,0.862401,0.823828,0.0,0.780454,0.000000,0.920393,0.772300,0.789752,0.812520,0.576338,0.469384,0.722062,basic,NaN,10220
2,1p_Eero_7_05122021_choreo1,2049,0.0,0.597563,0.783248,0.699501,0.400327,0.685925,0.415802,0.709093,0.416943,0.504014,0.587172,0.605324,0.450690,0.446109,0.327647,0.349077,0.527538,0.672309,0.301048,0.635318,0.187249,0.604922,0.316338,0.646947,0.442637,0.643845,0.375091,0.584295,0.388966,0.588487,0.471582,0.280982,0.897108,0.699714,0.701190,0.643996,0.663293,0.673902,...,0.487135,0.388075,0.505475,0.446063,0.617460,0.529752,0.651847,0.443344,0.660857,0.439839,0.679090,0.465854,0.842273,0.884259,0.832634,0.828490,0.296463,0.827820,0.903506,0.799872,0.728836,0.738614,0.831048,0.790879,0.681409,0.793449,0.811598,0.0,0.789451,0.000000,0.918237,0.773667,0.788074,0.807926,0.701255,0.589643,0.702901,basic,NaN,10220
3,1p_Eero_7_05122021_choreo1,2050,0.0,0.608388,0.777788,0.711030,0.430589,0.698785,0.439379,0.706593,0.371205,0.532974,0.584571,0.610423,0.484939,0.424878,0.317624,0.325344,0.471199,0.682223,0.325600,0.634568,0.184957,0.605214,0.314625,0.652792,0.435421,0.666939,0.435699,0.592740,0.396960,0.588800,0.468924,0.281593,0.897712,0.712318,0.696196,0.647180,0.659945,0.678305,...,0.485953,0.389963,0.503862,0.445615,0.616118,0.524253,0.655267,0.434850,0.664875,0.434421,0.685668,0.457193,0.790833,0.889678,0.826872,0.860735,0.408687,0.799390,0.944180,0.881825,0.737036,0.740039,0.819579,0.779000,0.691094,0.781239,0.813263,0.0,0.770174,0.000000,0.916075,0.794719,0.822523,0.826437,0.707142,0.631278,0.689080,basic,NaN,10220
4,1p_Eero_7_05122021_choreo1,2051,0.0,0.616613,0.765225,0.726336,0.468861,0.722256,0.483731,0.719604,0.363131,0.564945,0.603732,0.614661,0.514193,0.406888,0.295988,0.312158,0.427584,0.691927,0.351438,0.631523,0.177319,0.611003,0.320470,0.659510,0.436101,0.685409,0.475815,0.597620,0.401853,0.588521,0.466996,0.282205,0.898316,0.726546,0.691208,0.651907,0.660513,0.688574,...,0.485117,0.387622,0.502114,0.443139,0.614246

In [34]:
data_output_csv = os.path.join(data_output_path, "Data_train_orig_01_aug_x_xy.csv")
df_augmented.to_csv(data_output_csv)

In [26]:
df_perspective1.describe()

,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,figure_id
count,47542.000000,47542.0,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.00000
mean,2101.604728,0.0,0.412704,0.472340,0.405537,0.520015,0.391344,0.510650,0.409910,0.568325,0.401220,0.525748,0.425320,0.514722,0.439921,0.608204,0.421378,0.509591,0.378829,0.428738,0.370050,0.432045,0.391255,0.438492,0.397468,0.508689,0.403349,0.436621,0.390035,0.424921,0.399133,0.528680,0.391816,0.523784,0.421468,0.456963,0.396372,0.526892,0.432992,0.511428,0.399679,0.495953,0.399473,0.491822,0.403497,0.554969,0.404113,0.514560,0.406535,0.513624,0.402447,0.527816,0.675678,0.832992,0.798297,0.735279,0.671854,0.805461,0.737314,0.670927,0.693515,0.666842,0.784506,0.763711,0.663660,0.783899,0.754277,0.478777,0.490124,0.576999,0.594371,0.582188,0.580004,0.640954,0.594456,0.576488,0.650099,317.10170
std,1259.013850,0.0,0.164716,0.230172,0.165031,0.194759,0.154506,0.189209,0.172037,0.173462,0.160164,0.174615,0.182368,0.184718,0.179177,0.166831,0.165177,0.174696,0.142989,0.188225,0.137677,0.188729,0.132653,0.177319,0.134483,0.192702,0.152163,0.191667,0.142565,0.182648,0.150217,0.192882,0.187049,0.266690,0.161141,0.237608,0.171227,0.200904,0.177341,0.208223,0.139980,0.204536,0.143908,0.206968,0.155108,0.193510,0.136348,0.201153,0.136870,0.204888,0.138311,0.196911,0.381603,0.090391,0.106627,0.262938,0.300510,0.098560,0.253710,0.293835,0.088447,0.108436,0.081147,0.130296,0.108224,0.081825,0.132060,0.437355,0.435681,0.390745,0.387148,0.252700,0.262582,0.193792,0.245339,0.248304,0.188799,178.57796
min,18.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1087.000000,0.0,0.295495,0.273574,0.289704,0.379906,0.281838,0.371437,0.276055,0.447358,0.287007,0.413644,0.285081,0.375125,0.305513,0.501706,0.297816,0.393267,0.285142,0.284763,0.273

In [27]:
df_original.describe()

,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,figure_id
count,47542.000000,47542.0,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.000000,47542.00000
mean,2101.604728,0.0,0.524198,0.472340,0.501029,0.520015,0.488807,0.510650,0.495044,0.568325,0.496083,0.525748,0.525034,0.514722,0.524384,0.608204,0.526052,0.509591,0.493595,0.428738,0.481613,0.432045,0.505205,0.438492,0.496650,0.508689,0.523818,0.436621,0.511105,0.424921,0.493848,0.528680,0.486787,0.523784,0.541252,0.456963,0.488135,0.526892,0.539477,0.511428,0.502709,0.495953,0.502987,0.491822,0.492366,0.554969,0.503015,0.514560,0.506295,0.513624,0.498870,0.527816,0.675678,0.832992,0.798297,0.735279,0.671854,0.805461,0.737314,0.670927,0.693515,0.666842,0.784506,0.763711,0.663660,0.783899,0.754277,0.478777,0.490124,0.576999,0.594371,0.582188,0.580004,0.640954,0.594456,0.576488,0.650099,317.10170
std,1259.013850,0.0,0.200513,0.230172,0.196705,0.194759,0.191558,0.189209,0.204627,0.173462,0.194907,0.174615,0.212276,0.184718,0.215095,0.166831,0.204230,0.174696,0.184787,0.188225,0.178259,0.188729,0.164120,0.177319,0.165382,0.192702,0.197666,0.191667,0.189999,0.182648,0.183100,0.192882,0.221828,0.266690,0.201179,0.237608,0.205295,0.200904,0.215253,0.208223,0.173304,0.204536,0.177121,0.206968,0.185719,0.193510,0.164955,0.201153,0.165330,0.204888,0.170568,0.196911,0.381603,0.090391,0.106627,0.262938,0.300510,0.098560,0.253710,0.293835,0.088447,0.108436,0.081147,0.130296,0.108224,0.081825,0.132060,0.437355,0.435681,0.390745,0.387148,0.252700,0.262582,0.193792,0.245339,0.248304,0.188799,178.57796
min,18.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1087.000000,0.0,0.388280,0.273574,0.364711,0.379906,0.352573,0.371437,0.335059,0.447358,0.358168,0.413644,0.364662,0.375125,0.358388,0.501706,0.372652,0.393267,0.375036,0.284763,0.359

In [28]:
df_perspective1_norm.describe()

NameError: ignored